In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import sqlite3


class CaisseBoulangerie:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Caisse Boulangerie")
        self.window.geometry("1200x700")

        # Base de données
        self.conn = sqlite3.connect("vente_boulangerie.db")
        self.creer_table()

        # Produits prédéfinis (nom, prix)
        self.produits = {
            "Baguette": 100,
            "Croissant": 300,
            "Pain au chocolat": 400,
            "Pain aux raisins": 300,
            "Pain complet": 150,
            "Pain de campagne": 300,
            "Brioche": 300,
            "Pain aux céréales": 200
        }

        # Dictionnaire pour stocker les variables de quantité
        self.quantites = {}

        # Panier des achats
        self.panier = {}  # {nom: (quantite, prix_unitaire)}

        self.creer_interface()

    def creer_table(self):
        """Créer la table de ventes dans la base de données."""
        cursor = self.conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ventes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                produit TEXT,
                quantite INTEGER,
                prix_unitaire REAL,
                total REAL
            )
        """)
        self.conn.commit()

    def creer_interface(self):
        # Frame principale
        main_frame = ttk.Frame(self.window, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Frame gauche (produits)
        produits_frame = ttk.LabelFrame(main_frame, text="Produits", padding="10")
        produits_frame.grid(row=0, column=0, padx=5, pady=5, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Création des boutons et champs de quantité pour chaque produit
        for i, (nom, prix) in enumerate(self.produits.items()):
            product_frame = ttk.Frame(produits_frame)
            product_frame.grid(row=i, column=0, pady=5, sticky=(tk.W, tk.E))

            ttk.Label(product_frame, text=f"{nom} - {prix:.2f}FCFA").grid(row=0, column=0, padx=5)

            self.quantites[nom] = tk.StringVar(value="0")
            spinbox = ttk.Spinbox(
                product_frame,
                from_=0,
                to=100,
                width=5,
                textvariable=self.quantites[nom],
                command=lambda n=nom: self.mettre_a_jour_panier(n)
            )
            spinbox.grid(row=0, column=1, padx=5)
            spinbox.bind('<Return>', lambda e, n=nom: self.mettre_a_jour_panier(n))
            spinbox.bind('<FocusOut>', lambda e, n=nom: self.mettre_a_jour_panier(n))

        # Frame droite (panier et total)
        panier_frame = ttk.LabelFrame(main_frame, text="Panier", padding="10")
        panier_frame.grid(row=0, column=1, padx=5, pady=5, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Liste du panier
        self.panier_tree = ttk.Treeview(panier_frame, columns=('Quantité', 'Prix unitaire', 'Total'))
        self.panier_tree.heading('Quantité', text='Quantité')
        self.panier_tree.heading('Prix unitaire', text='Prix unitaire')
        self.panier_tree.heading('Total', text='Total')
        self.panier_tree.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Total
        self.total_var = tk.StringVar(value="Total: 0.00FCFA")
        total_label = ttk.Label(panier_frame, textvariable=self.total_var, font=('Arial', 16, 'bold'))
        total_label.grid(row=1, column=0, columnspan=2, pady=10)

        # Paiement
        paiement_frame = ttk.LabelFrame(panier_frame, text="Paiement", padding="10")
        paiement_frame.grid(row=2, column=0, columnspan=2, sticky=(tk.W, tk.E))

        ttk.Label(paiement_frame, text="Montant reçu:").grid(row=0, column=0, padx=5)
        self.montant_recu_var = tk.StringVar()
        montant_recu_entry = ttk.Entry(paiement_frame, textvariable=self.montant_recu_var)
        montant_recu_entry.grid(row=0, column=1, padx=5)
        montant_recu_entry.bind('<Return>', self.calculer_monnaie)
        montant_recu_entry.bind('<KeyRelease>', self.calculer_monnaie)

        self.monnaie_var = tk.StringVar(value="Monnaie à rendre: 0.00FCFA")
        ttk.Label(paiement_frame, textvariable=self.monnaie_var, font=('Arial', 12)).grid(row=1, column=0, columnspan=2, pady=5)

        # Boutons
        ttk.Button(panier_frame, text="Nouvelle vente", command=self.nouvelle_vente).grid(row=3, column=0, pady=10)
        ttk.Button(panier_frame, text="Imprimer ticket", command=self.imprimer_ticket).grid(row=3, column=1, pady=10)
        ttk.Button(panier_frame, text="Valider Vente", command=self.valider_vente).grid(row=2, column=0, columnspan=2, pady=10)
        ttk.Button(main_frame, text="Afficher Ventes", command=self.afficher_ventes).grid(row=2, column=1, pady=10)

    def mettre_a_jour_panier(self, nom_produit):
        try:
            quantite = int(self.quantites[nom_produit].get())
            if quantite < 0:
                raise ValueError
        except ValueError:
            self.quantites[nom_produit].set("0")
            quantite = 0

        if quantite > 0:
            self.panier[nom_produit] = (quantite, self.produits[nom_produit])
        else:
            self.panier.pop(nom_produit, None)

        self.rafraichir_affichage()

    def rafraichir_affichage(self):
        for item in self.panier_tree.get_children():
            self.panier_tree.delete(item)

        total = 0
        for nom, (quantite, prix) in self.panier.items():
            sous_total = quantite * prix
            total += sous_total
            self.panier_tree.insert('', 'end', text=nom, values=(quantite, f"{prix:.2f}FCFA", f"{sous_total:.2f}FCFA"))

        self.total_var.set(f"Total: {total:.2f}FCFA")
        self.calculer_monnaie()

    def calculer_monnaie(self, event=None):
        try:
            total = float(self.total_var.get().split(":")[1].replace("FCFA", "").strip())
            recu = float(self.montant_recu_var.get() or 0)
            monnaie = recu - total
            if monnaie >= 0:
                self.monnaie_var.set(f"Monnaie à rendre: {monnaie:.2f}FCFA")
            else:
                self.monnaie_var.set("Montant insuffisant")
        except ValueError:
            self.monnaie_var.set("Montant invalide")

    def nouvelle_vente(self):
        for var in self.quantites.values():
            var.set("0")
        self.panier.clear()
        self.montant_recu_var.set("")
        self.rafraichir_affichage()

    def imprimer_ticket(self):
        if not self.panier:
            messagebox.showwarning("Attention", "Le panier est vide")
            return

        ticket = tk.Toplevel(self.window)
        ticket.title("Ticket")
        ticket.geometry("300x400")

        text = tk.Text(ticket, height=20, width=35)
        text.pack(padx=10, pady=10)

        text.insert('1.0', "=== BOULANGERIE ===\n\n")
        text.insert('end', f"Date: {datetime.now().strftime('%d/%m/%Y %H:%M')}\n\n")
        total = 0
        for nom, (quantite, prix) in self.panier.items():
            sous_total = quantite * prix
            text.insert('end', f"{nom}\n")
            text.insert('end', f"{quantite} x {prix:.2f}FCFA = {sous_total:.2f}FCFA\n")
            total += sous_total

            self.sauvegarder_vente(nom, quantite, prix, sous_total)

        text.insert('end', f"\nTotal: {self.total_var.get().split(':')[1]}\n")
        monnaie = self.monnaie_var.get()
        if "rendre" in monnaie:
            text.insert('end', f"\n{monnaie}\n")

        text.config(state='disabled')

    def valider_vente(self):
        if not self.panier:
            messagebox.showwarning("Attention", "Le panier est vide.")
            return

        for nom, (quantite, prix) in self.panier.items():
            sous_total = quantite * prix
            self.sauvegarder_vente(nom, quantite, prix, sous_total)

        messagebox.showinfo("Succès", "Les ventes ont été validées avec succès.")
        self.nouvelle_vente()

    def sauvegarder_vente(self, produit, quantite, prix_unitaire, total):
        date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cursor = self.conn.cursor()
        cursor.execute("""
            INSERT INTO ventes (date, produit, quantite, prix_unitaire, total)
            VALUES (?, ?, ?, ?, ?)
        """, (date, produit, quantite, prix_unitaire, total))
        self.conn.commit()

    def afficher_ventes(self):
        fenetre_ventes = tk.Toplevel(self.window)
        fenetre_ventes.title("Historique des ventes")
        fenetre_ventes.geometry("600x400")

        tree = ttk.Treeview(fenetre_ventes, columns=("Date", "Produit", "Quantité", "Prix unitaire", "Total"), show='headings')
        tree.heading("Date", text="Date")
        tree.heading("Produit", text="Produit")
        tree.heading("Quantité", text="Quantité")
        tree.heading("Prix unitaire", text="Prix unitaire")
        tree.heading("Total", text="Total")
        tree.pack(fill=tk.BOTH, expand=True)

        cursor = self.conn.cursor()
        cursor.execute("SELECT date, produit, quantite, prix_unitaire, total FROM ventes")
        for row in cursor.fetchall():
            tree.insert("", "end", values=row)

    def run(self):
        self.window.mainloop()


if __name__ == "__main__":
    caisse = CaisseBoulangerie()
    caisse.run()
